In [ ]:
%%writefile prod_invent.py
from dataclasses import dataclass, field
from typing import List, Optional


class InvalidDiscountError(Exception):
    """Custom exception for invalid discount."""
    pass


@dataclass
class Category:
    name: str
    description: Optional[str] = ""

    def __str__(self):
        return f"{self.name}: {self.description}"


@dataclass
class Product:
    name: str
    price: float
    quantity: int
    discount: Optional[float] = 0.0  
    category: Category = field(default_factory=lambda: Category(name="General", description="General Category"))
    tags: List[str] = field(default_factory=list)

    def __post_init__(self):
        """Post initialization checks and validations."""
        if self.price < 0:
            raise ValueError(f"Price cannot be negative: {self.price}")
        if self.quantity < 0:
            raise ValueError(f"Quantity cannot be negative: {self.quantity}")
        if self.discount < 0 or self.discount > 100:
            raise InvalidDiscountError(f"Invalid discount: {self.discount}. Must be between 0 and 100.")

    def total_price(self) -> float:
        """Calculate the total price after applying the discount."""
        return self.price * self.quantity * (1 - self.discount / 100)

    def apply_discount(self, discount: float) -> None:
        """Apply a discount to the product."""
        if discount < 0 or discount > 100:
            raise InvalidDiscountError(f"Discount must be between 0 and 100.")
        self.discount = discount

    def update_quantity(self, change: int) -> None:
        """Update the product quantity by adding a change (can be negative)."""
        if self.quantity + change < 0:
            raise ValueError("Quantity cannot go below zero.")
        self.quantity += change

    def add_tags(self, *new_tags: str) -> None:
        """Add tags to the product."""
        self.tags.extend(new_tags)

    def apply_bulk_discount(self, bulk_discount_threshold: int, discount: float) -> None:
        """Apply a bulk discount if quantity exceeds threshold."""
        if self.quantity >= bulk_discount_threshold:
            self.apply_discount(discount)


@dataclass
class Inventory:
    products: List[Product] = field(default_factory=list)

    def add_product(self, product: Product) -> None:
        """Add a product to the inventory."""
        self.products.append(product)

    def get_product_by_name(self, name: str) -> Optional[Product]:
        """Find a product by its name."""
        return next((product for product in self.products if product.name == name), None)

    def total_inventory_value(self) -> float:
        """Calculate the total value of the inventory."""
        return sum(product.total_price() for product in self.products)


Overwriting prod_invent.py


Testing

In [106]:
%%writefile test_prod_invent.py
from prod_invent import InvalidDiscountError, Category, Product, Inventory
import pytest


def test_category():
    category = Category(name = 'Electronics', description = "device and gadgets")
    assert category.name == 'Electronics'
    assert category.description == 'device and gadgets'

def test_product():
    category = Category(name = 'Electronics', description = "device and gadgets")
    product =  Product(name =  'Mobile', price = 3000.0, quantity = 3, discount = 0.0, tags = ['IOT', 'AI'])
    assert product.name == 'Mobile'
    assert product.price == 3000.0
    assert product.quantity == 3
    assert product.discount == 0.0
    assert product.tags == ['IOT', 'AI']

def test_total_price_wthout_discount():
    product =  Product(name =  'Mobile', price = 3000.0, quantity = 3, tags = ['IOT', 'AI'])
    total = product.total_price()
    assert total == 9000.0

#def test_total_price_with_discount():
#    product =  Product(name = 'Mobile', price =  3000.0, discount =  10.0, quantity = 3, tags = ['IOT', 'AI'])
#    total = product.total_price()
#   assert total ==  8100.0

    #or

def test_total_price_with_discount():
    product =  Product(name = 'Mobile', price =  3000.0, quantity = 3, tags = ['IOT', 'AI'])
    discount = product.apply_discount(10.0)
    total =  product.total_price()
    assert total ==  8100.0

def test_update_quantity():
    product = Product(name = 'Mobile', price =  3000.0, quantity = 3, tags = ['IOT', 'AI'])
    quantity = product.update_quantity(5)
    total =  product.total_price()
    assert product.quantity == 8
    assert total == 24000.0

def test_add_tags():
    product = Product(name = 'Mobile', price =  3000.0, quantity = 3, tags = ['IOT', 'AI'])
    product.add_tags('smart devices')
    assert product.tags == ['IOT', 'AI', 'smart devices']

def test_bulk_discount():
    product = Product(name = 'Mobile', price =  3000.0, quantity = 3, tags = ['IOT', 'AI'])
    product.apply_bulk_discount(4, 10)
    #product.apply_discount(10)
    total =  product.total_price()
    assert total == 9000.0

def test_inventory_management():
    inventory = Inventory()
    category = Category(name="Furniture", description="Home furnishings")
    product1 = Product(name="Sofa", price=700.0, quantity=3, category=category)
    product2 = Product(name="Chair", price=150.0, quantity=10, category=category)

    inventory.add_product(product1)
    inventory.add_product(product2)

    # Test getting a product by name
    retrieved_product = inventory.get_product_by_name("Sofa")
    assert retrieved_product == product1

    # Test calculating the total inventory value
    total_value = inventory.total_inventory_value()
    assert total_value == (700.0 * 3) + (150.0 * 10)


def test_invalid_price_and_quantity():
    """Test that invalid price and quantity raise errors."""
    with pytest.raises(ValueError):
        Product(name="Table", price=-100.0, quantity=5)  # Invalid price

    with pytest.raises(ValueError):
        Product(name="Table", price=100.0, quantity=-5)  # Invalid quantity


Overwriting test_prod_invent.py


In [107]:
!pytest test_prod_invent.py

============================= test session starts ==============================
platform linux -- Python 3.8.10, pytest-8.3.5, pluggy-1.5.0
rootdir: /home/amit.pathak
collected 9 items                                                              

test_prod_invent.py .........                                            [100%]

============================== 9 passed in 0.03s ===============================


Using fixture

In [ ]:
%%writefile test_prod_invent1.py
from prod_invent import InvalidDiscountError, Category, Product, Inventory
import pytest

@pytest.fixture
def sample_product():
    return Product(name =  'Mobile', price = 3000.0, quantity = 3, discount = 0.0, tags = ['IOT', 'AI'])

@pytest.fixture
def sample_category():
    return Category(name = 'Electronics', description = "device and gadgets")

def test_category(sample_category):
    #category = Category(name = 'Electronics', description = "device and gadgets")
    assert sample_category.name == 'Electronics'
    assert sample_category.description == 'device and gadgets'

def test_product(sample_category, sample_product):
    #category = Category(name = 'Electronics', description = "device and gadgets")
    #product =  Product(name =  'Mobile', price = 3000.0, quantity = 3, discount = 0.0, tags = ['IOT', 'AI'])
    assert sample_product.name == 'Mobile'
    assert sample_product.price == 3000.0
    assert sample_product.quantity == 3
    assert sample_product.discount == 0.0
    assert sample_product.tags == ['IOT', 'AI']

def test_total_price_wthout_discount(sample_product):
    #product =  Product(name =  'Mobile', price = 3000.0, quantity = 3, tags = ['IOT', 'AI'])
    total = sample_product.total_price()
    assert total == 9000.0

#def test_total_price_with_discount():
#    product =  Product(name = 'Mobile', price =  3000.0, discount =  10.0, quantity = 3, tags = ['IOT', 'AI'])
#    total = product.total_price()
#   assert total ==  8100.0

    #or

def test_total_price_with_discount(sample_product):
    #product =  Product(name = 'Mobile', price =  3000.0, quantity = 3, tags = ['IOT', 'AI'])
    discount = sample_product.apply_discount(10.0)
    total =  sample_product.total_price()
    assert total ==  8100.0

def test_update_quantity(sample_product):
    #product = Product(name = 'Mobile', price =  3000.0, quantity = 3, tags = ['IOT', 'AI'])
    quantity = sample_product.update_quantity(5)
    total =  sample_product.total_price()
    assert sample_product.quantity == 8
    assert total == 24000.0

def test_add_tags(sample_product):
    #product = Product(name = 'Mobile', price =  3000.0, quantity = 3, tags = ['IOT', 'AI'])
    sample_product.add_tags('smart devices')
    assert sample_product.tags == ['IOT', 'AI', 'smart devices']

def test_bulk_discount(sample_product):
    #product = Product(name = 'Mobile', price =  3000.0, quantity = 3, tags = ['IOT', 'AI'])
    sample_product.apply_bulk_discount(4, 10)
    #product.apply_discount(10)
    total =  sample_product.total_price()
    assert total == 9000.0


def test_inventory_management(sample_product, sample_category ):
    inventory = Inventory()
    #category = Category(name="Furniture", description="Home furnishings")
    #product1 = Product(name="Sofa", price=700.0, quantity=3, category=category)
    #product2 = Product(name="Chair", price=150.0, quantity=10, category=category)

    inventory.add_product(sample_product)
    #inventory.add_product(product2)

    # Test getting a product by name
    retrieved_product = inventory.get_product_by_name("Mobile")
    assert retrieved_product == sample_product

    # Test calculating the total inventory value
    total_value = inventory.total_inventory_value()
    assert total_value == 9000.0


def test_invalid_price_and_quantity():
    """Test that invalid price and quantity raise errors."""
    with pytest.raises(ValueError):
        Product(name="Table", price=-100.0, quantity=5)  # Invalid price

    with pytest.raises(ValueError):
        Product(name="Table", price=100.0, quantity=-5)  # Invalid quantity


Overwriting test_prod_invent1.py


In [105]:
!pytest test_prod_invent1.py

============================= test session starts ==============================
platform linux -- Python 3.8.10, pytest-8.3.5, pluggy-1.5.0
rootdir: /home/amit.pathak
collected 9 items                                                              

test_prod_invent1.py .........                                           [100%]

============================== 9 passed in 0.03s ===============================


Using Parameterization

In [ ]:
def test_category():
    category = Category(name = 'Electronics', description = "device and gadgets")
    assert category.name == 'Electronics'
    assert category.description == 'device and gadgets'

In [116]:

%%writefile test_prod_invent2.py
import pytest
from prod_invent import InvalidDiscountError, Category, Product, Inventory


@pytest.mark.parametrize(
    "name, description",
    [
        ('mobile', 'this product is categories in devices'),
        ('headphone' , 'this product is categories in gadgets'),
        ('mouse' , 'this product is categories in gadgets')
    ]
)

def test_category(name, description):
    category =  Category(name = name, description  =  description)
    assert category.name == name
    assert category.description == description

Overwriting test_prod_invent2.py


In [117]:
!pytest test_prod_invent2.py

============================= test session starts ==============================
platform linux -- Python 3.8.10, pytest-8.3.5, pluggy-1.5.0
rootdir: /home/amit.pathak
collected 3 items                                                              

test_prod_invent2.py ...                                                 [100%]

============================== 3 passed in 0.02s ===============================


In [ ]:
%%writefile test_prod_invent2.py
from prod_invent import InvalidDiscountError, Category, Product, Inventory
import pytest

@pytest.mark.parameterize(
    "name, price, quantity, discount, tags",
    [
        ('Mobile', 3000.0, 3, 0.0, ['IOT', 'AI']),
        ('Laptop', 5000.0, 2, 10.0, ['AI', 'ML']),
        ('Tablet', 1500.0, 5, 5.0, ['AI', 'IOT']),
    ]
)

def test_product(name, price, quantity, discount, tags):
    product = Product(name =  name, price = price, quantity)